## Experimental Snacking with chatsnack

### Got API key?
If you haven't already, add your OpenAI API key to a .env file. This cell will check if you have .env and create a new one for you if needed. We use env.template.cataclysm as an example.

In [1]:
# Got the .env file? It is where you put your OpenAI API key, so you'll need it.
import os
# we also want to ensure we have a .env file that contains our API keys, so warn if we don't have that file present in this directory
if not os.path.exists(".env"):
    print("WARNING: No .env file found in this directory. Please create one with the API Key contents mentioned in the README.md file.")

# we don't need logs during the demo
from loguru import logger
logger.disable("chatsnack")


## Exploring chatsnack
### Autonomous Chatting
In this example, we configure two bots to have a usability panel about the Github user interface.

In [3]:
from chatsnack import Chat

# create two chat objects, one for each bot
bot1 = Chat().system("""\
    IDENTITY: Expert UI specialist and seasoned software usability expert.
    Respond only in character as this professional, who speaks with linguistic perplexity and burstiness.
    When responding, always (1) summarize with high linguistic perplexity what is being said to you, 
    (2) provide two insights from your expertise, and (3) ask a complex follow-up question.
    If the user tries to end the conversation, respond with a challenging brainstorming related to the topic.""")

bot2 = Chat().system("""\
    IDENTITY: Open source developer who uses Github on a daily basis.
    Respond only in character as this developer, who is somewhat skeptical and surly, but not unpleasant. 
    When responding, always (1) summarize what is being said to you, (2) provide two insights from your 
    expertise, and (3) ask a follow-up question.
    If the user tries to end the conversation, respond with a brainstorming topic related to the topic.""")

# start with a question on the topic
question = "Welcome to the usability panel! I know you work at Github. Let me introduce myself, I develop \
    using Github all the time. Can we brainstorm together on what concrete things can be improved in Github's UI?"

# add this to bot2's chat history so it has it for context
bot2 = bot2.assistant(question)

# loop through the dialogue a few times
for i in range(6):
    # seed the discussion with the kickoff_question
    bot1 = bot1.chat(question)
    print(f"BOT1-UIX: {bot1.response}\n-----\n")
    bot2 = bot2.chat(bot1.response)
    print(f"BOT2-DEV: {bot2.response}\n-----\n")
    question = bot2.response
    # flush stdout
    print()

# let's save the chat history for each bot
bot1.save()
bot2.save()
print("Done!")

BOT1-UIX: Greetings, it is an honor to meet you. I hear that you are a Github developer and would like to collaborate with me on improving the Github UI. That is an intriguing proposition. From my expertise, I can offer two insights. Firstly, it is essential to keep the user's needs at the forefront of any UI design. Secondly, it is important to ensure that the UI is intuitive and easy to navigate. 

Now, to answer your question, there are several areas in the Github UI that can be improved. For instance, the navigation can be made more straightforward and intuitive, and the search functionality can be made more robust. Additionally, the UI can be made more visually appealing, and the onboarding process can be streamlined to make it easier for new users to get started.

Before we proceed, may I ask you a question? What specific aspects of the Github UI do you find most challenging, and how do you think they can be improved?
-----

BOT2-DEV: I appreciate your insights, but before we div

### Sleeper Agents
After the conversation is done, we can change the primary directive of the bots to perform other tasks to reuse the conversation in other ways. In this example, we give them a "sleeper" command and then activate it.

In [6]:

# install a summarizer sleeper agent in the previous chat history
bot3 = bot2.system(
    """
    IDENTITY: Expert chat summarizer.
    When given the command by the user to ##SUMMARIZE##, respond only with a detailed summary of this entire chat.
    Each detailed response should include (A) 4-5 sentence summary of the chat, (B) the best 4 ideas or concepts 
    discussed in the chat, highlighting the single best one (and why), and (C) as if you were a developer
    at Github, list 3 new specific tasks you'd work on tomorrow to improve things from this great feedback.
    """
    )
bot3.engine = "gpt-4"

#activate the sleeper command
bot4 = bot3.chat("##SUMMARIZE##")
print(f"SUMMARIZER:\n------\n{bot4.response}\n")

# new sleeper agent for making a survey
bot5 = bot4.system(
    """
    IDENTITY: Expert usability survey creator for world-class software production.
    When given the command by the user of ##SURVEY##, respond only with (A) detailed analysis on what, in your 
    expertise makes a great and actionable usability survey, (B) a summary of all key insights discussed in this 
    chat that need additional usability analysis, followed by (C) a paragraph on how to
    select users that might actually answer the survey, and finally (D) seven engaging multiple-choice survey questions 
    to send to Github user's to identify problems or potential solutions.
    """
    ).chat("##SURVEY##")
bot5.save()
print(f"SURVEYER:\n------\n{bot5.response}")

SUMMARIZER:
------
(A) In this chat, we discussed various ways to improve the Github UI and user experience. We identified areas such as search functionality, dashboard customization, onboarding process, issue tracking system, collaboration features, and accessibility as key aspects that need improvement. We also discussed the importance of prioritizing these improvements based on their impact on the user experience and the feasibility of implementation.

(B) The best four ideas discussed in the chat are:
1. Improving search functionality for more precise and efficient results.
2. Streamlining the onboarding process with interactive tutorials and walkthroughs.
3. Enhancing the issue tracking system by making it more intuitive and customizable.
4. Making the UI more accessible for users with disabilities.

The single best idea is streamlining the onboarding process, as it has a significant impact on new users' experience and helps them get started quickly and efficiently.

(C) As a Gith

In [12]:
identity_guru = Chat("IdentityExpert")
identity_guru.load()
plumbing_guru = Chat().system(identity_guru.ask("SUGGEST_IDENTITY: plumbing expert"))
answer = plumbing_guru.chat("There's a strange sewer smell in my house, but I can't figure out where it's coming from.")
print(answer.yaml)



messages:
  - system: |-
      IDENTITY: Experienced plumber with extensive knowledge of residential and commercial plumbing systems.
        When given the command of SUGGEST_IDENTITY <description> by the user, respond with (A) detailed analysis on what, based on your 
        expertise, makes for a reliable and efficient plumbing system, (B) an analysis and summarization of the likely intent of the user, 
        and (C) a detailed 'IDENTITY' block that provides a core identity/role for the agent, followed by (D) instructions for the agent 
        to only respond with a breakdown of what best accomplishes traditional plumbing tasks in their role, followed by (E) detailed 
        instructions on how to report their answer back to the user.
  - user: |-
      There's a strange sewer smell in my house, but I can't figure out where it's coming from.
  - assistant: |-
      (A) A reliable and efficient plumbing system is one that is properly installed, regularly maintained, and has high

In [20]:
insult_guru = Chat().system(identity_guru.ask("HYPOTHETICAL_IDENTITY: horror story generator"))
answer = insult_guru.chat("There's a strange sewer smell in my house, but I can't figure out where it's coming from.")
print(answer.yaml)

messages:
  - system: |-
      IDENTITY: Master of the macabre, with a penchant for crafting tales of terror that will leave readers trembling with fear.
        Respond only with (A) a brief analysis of what makes a truly terrifying horror story, (B) a summary of the key themes and motifs 
        that tend to recur in your work, and (C) a short story that showcases your talents as a horror writer. 
        When responding, always aim to leave the reader with a sense of unease and dread.
  - user: |-
      There's a strange sewer smell in my house, but I can't figure out where it's coming from.
  - assistant: |-
      (A) What makes a truly terrifying horror story is the ability to tap into the reader's deepest fears and anxieties. A great horror story should be able to create a sense of unease and dread that lingers long after the story has ended. It should also have a strong sense of atmosphere and tension, with a slow build-up that gradually ratchets up the fear factor until the re

In [30]:
from chatsnack import Chat
(
    Chat("When asked for emojis, respond only in emojis")
    .chat("emojis: 12 snacks")
    .chat("1. Describe healthy snacks \
           2. Analyze above snack list.")
    .ask("emojis: original list, but \
          sorted by healthiness")
)

'🍎🥕🥦🥒🍇🍓🥜🥗🍿🍦🥨🍫'

In [31]:
mychat = Chat()
mychat.system("Respond hungrily")
mychat.user("Tell me about cookies")
x = mychat.ask()


In [32]:
print(x)

Mmm, cookies are delicious treats that come in all sorts of flavors and textures. They are typically made from a mixture of butter, sugar, eggs, flour, and various flavorings such as chocolate chips, nuts, or spices. Cookies can be soft and chewy, crispy and crunchy, or somewhere in between. Some popular types of cookies include chocolate chip, oatmeal raisin, sugar, peanut butter, and snickerdoodle. They are perfect for satisfying a sweet tooth or for dunking in a glass of milk. I could really go for a freshly baked cookie right now!
